In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [5]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.288069
batch 1: loss 2.275034
batch 2: loss 2.099749
batch 3: loss 2.049044
batch 4: loss 1.909341
batch 5: loss 1.741766
batch 6: loss 1.557925
batch 7: loss 1.384226
batch 8: loss 1.243947
batch 9: loss 0.812217
batch 10: loss 1.059941
batch 11: loss 0.691393
batch 12: loss 0.785268
batch 13: loss 0.928908
batch 14: loss 0.573810
batch 15: loss 0.637255
batch 16: loss 0.608649
batch 17: loss 0.665204
batch 18: loss 0.528825
batch 19: loss 0.362198
batch 20: loss 0.709469
batch 21: loss 0.472980
batch 22: loss 0.574443
batch 23: loss 0.460927
batch 24: loss 0.430071
batch 25: loss 0.259138
batch 26: loss 0.378769
batch 27: loss 0.447599
batch 28: loss 0.538117
batch 29: loss 0.342883
batch 30: loss 0.383705
batch 31: loss 0.537025
batch 32: loss 0.383701
batch 33: loss 0.351175
batch 34: loss 0.525767
batch 35: loss 0.234480
batch 36: loss 0.289584
batch 37: loss 0.392871
batch 38: loss 0.303420
batch 39: loss 0.253252
batch 40: loss 0.418348
batch 41: loss 0.392523
ba

batch 334: loss 0.008317
batch 335: loss 0.207333
batch 336: loss 0.046464
batch 337: loss 0.050880
batch 338: loss 0.006236
batch 339: loss 0.074557
batch 340: loss 0.041513
batch 341: loss 0.040066
batch 342: loss 0.006323
batch 343: loss 0.011653
batch 344: loss 0.186266
batch 345: loss 0.014273
batch 346: loss 0.084094
batch 347: loss 0.012882
batch 348: loss 0.038680
batch 349: loss 0.183137
batch 350: loss 0.111708
batch 351: loss 0.078966
batch 352: loss 0.017764
batch 353: loss 0.125066
batch 354: loss 0.193970
batch 355: loss 0.065448
batch 356: loss 0.054140
batch 357: loss 0.111090
batch 358: loss 0.021726
batch 359: loss 0.108191
batch 360: loss 0.048855
batch 361: loss 0.002790
batch 362: loss 0.085090
batch 363: loss 0.044191
batch 364: loss 0.185334
batch 365: loss 0.044173
batch 366: loss 0.013094
batch 367: loss 0.033795
batch 368: loss 0.064099
batch 369: loss 0.156852
batch 370: loss 0.016580
batch 371: loss 0.124277
batch 372: loss 0.100577
batch 373: loss 0.075197


batch 662: loss 0.026535
batch 663: loss 0.019638
batch 664: loss 0.104305
batch 665: loss 0.152286
batch 666: loss 0.021134
batch 667: loss 0.031701
batch 668: loss 0.112463
batch 669: loss 0.063287
batch 670: loss 0.068330
batch 671: loss 0.046353
batch 672: loss 0.001702
batch 673: loss 0.035787
batch 674: loss 0.040344
batch 675: loss 0.152805
batch 676: loss 0.075750
batch 677: loss 0.046115
batch 678: loss 0.138114
batch 679: loss 0.014468
batch 680: loss 0.096349
batch 681: loss 0.063561
batch 682: loss 0.036159
batch 683: loss 0.070894
batch 684: loss 0.041304
batch 685: loss 0.120135
batch 686: loss 0.024463
batch 687: loss 0.023755
batch 688: loss 0.043421
batch 689: loss 0.053023
batch 690: loss 0.026532
batch 691: loss 0.062218
batch 692: loss 0.254514
batch 693: loss 0.040188
batch 694: loss 0.047705
batch 695: loss 0.039991
batch 696: loss 0.060895
batch 697: loss 0.009094
batch 698: loss 0.014843
batch 699: loss 0.178140
batch 700: loss 0.090283
batch 701: loss 0.022872


batch 991: loss 0.004048
batch 992: loss 0.006850
batch 993: loss 0.082969
batch 994: loss 0.039092
batch 995: loss 0.092622
batch 996: loss 0.012395
batch 997: loss 0.165374
batch 998: loss 0.107642
batch 999: loss 0.002559
batch 1000: loss 0.004667
batch 1001: loss 0.155206
batch 1002: loss 0.070679
batch 1003: loss 0.036425
batch 1004: loss 0.010927
batch 1005: loss 0.007845
batch 1006: loss 0.009424
batch 1007: loss 0.028085
batch 1008: loss 0.020225
batch 1009: loss 0.023775
batch 1010: loss 0.048268
batch 1011: loss 0.016241
batch 1012: loss 0.135681
batch 1013: loss 0.116682
batch 1014: loss 0.041441
batch 1015: loss 0.058456
batch 1016: loss 0.003086
batch 1017: loss 0.063395
batch 1018: loss 0.164004
batch 1019: loss 0.025848
batch 1020: loss 0.027202
batch 1021: loss 0.001997
batch 1022: loss 0.146364
batch 1023: loss 0.014449
batch 1024: loss 0.063923
batch 1025: loss 0.007350
batch 1026: loss 0.010678
batch 1027: loss 0.002330
batch 1028: loss 0.006809
batch 1029: loss 0.00

batch 1308: loss 0.062843
batch 1309: loss 0.005634
batch 1310: loss 0.047976
batch 1311: loss 0.004278
batch 1312: loss 0.050132
batch 1313: loss 0.013806
batch 1314: loss 0.066574
batch 1315: loss 0.002433
batch 1316: loss 0.054371
batch 1317: loss 0.009022
batch 1318: loss 0.058974
batch 1319: loss 0.011567
batch 1320: loss 0.013834
batch 1321: loss 0.002023
batch 1322: loss 0.060457
batch 1323: loss 0.270751
batch 1324: loss 0.001546
batch 1325: loss 0.010286
batch 1326: loss 0.010578
batch 1327: loss 0.002358
batch 1328: loss 0.086659
batch 1329: loss 0.000293
batch 1330: loss 0.032119
batch 1331: loss 0.045333
batch 1332: loss 0.185040
batch 1333: loss 0.077772
batch 1334: loss 0.044456
batch 1335: loss 0.050546
batch 1336: loss 0.009201
batch 1337: loss 0.012477
batch 1338: loss 0.000857
batch 1339: loss 0.012351
batch 1340: loss 0.018702
batch 1341: loss 0.045795
batch 1342: loss 0.064958
batch 1343: loss 0.014438
batch 1344: loss 0.044098
batch 1345: loss 0.033157
batch 1346: 

...

batch 4470: loss 0.025418
batch 4471: loss 0.007886
batch 4472: loss 0.026544
batch 4473: loss 0.001414
batch 4474: loss 0.015557
batch 4475: loss 0.026076
batch 4476: loss 0.009303
batch 4477: loss 0.018872
batch 4478: loss 0.000689
batch 4479: loss 0.000200
batch 4480: loss 0.014292
batch 4481: loss 0.007289
batch 4482: loss 0.000041
batch 4483: loss 0.000430
batch 4484: loss 0.020902
batch 4485: loss 0.000021
batch 4486: loss 0.008828
batch 4487: loss 0.003464
batch 4488: loss 0.000554
batch 4489: loss 0.079523
batch 4490: loss 0.000770
batch 4491: loss 0.011506
batch 4492: loss 0.007760
batch 4493: loss 0.000248
batch 4494: loss 0.000451
batch 4495: loss 0.061213
batch 4496: loss 0.013977
batch 4497: loss 0.004457
batch 4498: loss 0.002034
batch 4499: loss 0.014927
batch 4500: loss 0.003082
batch 4501: loss 0.006402
batch 4502: loss 0.052745
batch 4503: loss 0.000200
batch 4504: loss 0.109371
batch 4505: loss 0.001876
batch 4506: loss 0.003504
batch 4507: loss 0.042394
batch 4508: 

batch 4785: loss 0.001840
batch 4786: loss 0.000158
batch 4787: loss 0.001805
batch 4788: loss 0.000045
batch 4789: loss 0.094105
batch 4790: loss 0.000137
batch 4791: loss 0.003084
batch 4792: loss 0.000967
batch 4793: loss 0.004233
batch 4794: loss 0.067752
batch 4795: loss 0.001307
batch 4796: loss 0.013158
batch 4797: loss 0.000585
batch 4798: loss 0.003609
batch 4799: loss 0.000980
batch 4800: loss 0.004780
batch 4801: loss 0.025142
batch 4802: loss 0.002537
batch 4803: loss 0.000632
batch 4804: loss 0.058803
batch 4805: loss 0.024767
batch 4806: loss 0.020873
batch 4807: loss 0.000925
batch 4808: loss 0.027022
batch 4809: loss 0.059112
batch 4810: loss 0.002531
batch 4811: loss 0.011968
batch 4812: loss 0.000283
batch 4813: loss 0.118986
batch 4814: loss 0.001041
batch 4815: loss 0.000681
batch 4816: loss 0.001365
batch 4817: loss 0.036293
batch 4818: loss 0.008764
batch 4819: loss 0.067614
batch 4820: loss 0.006875
batch 4821: loss 0.001261
batch 4822: loss 0.004806
batch 4823: 

batch 5102: loss 0.002957
batch 5103: loss 0.024310
batch 5104: loss 0.000254
batch 5105: loss 0.026761
batch 5106: loss 0.050742
batch 5107: loss 0.003145
batch 5108: loss 0.001524
batch 5109: loss 0.065748
batch 5110: loss 0.000323
batch 5111: loss 0.007215
batch 5112: loss 0.000754
batch 5113: loss 0.005567
batch 5114: loss 0.002163
batch 5115: loss 0.061284
batch 5116: loss 0.001020
batch 5117: loss 0.000198
batch 5118: loss 0.000967
batch 5119: loss 0.001254
batch 5120: loss 0.023121
batch 5121: loss 0.065456
batch 5122: loss 0.076386
batch 5123: loss 0.046009
batch 5124: loss 0.005161
batch 5125: loss 0.000426
batch 5126: loss 0.002577
batch 5127: loss 0.001250
batch 5128: loss 0.005743
batch 5129: loss 0.000645
batch 5130: loss 0.000907
batch 5131: loss 0.049963
batch 5132: loss 0.002432
batch 5133: loss 0.001970
batch 5134: loss 0.004354
batch 5135: loss 0.000274
batch 5136: loss 0.018302
batch 5137: loss 0.002987
batch 5138: loss 0.005228
batch 5139: loss 0.001751
batch 5140: 

batch 5418: loss 0.002204
batch 5419: loss 0.003000
batch 5420: loss 0.000312
batch 5421: loss 0.008645
batch 5422: loss 0.002648
batch 5423: loss 0.003473
batch 5424: loss 0.000925
batch 5425: loss 0.000113
batch 5426: loss 0.001573
batch 5427: loss 0.073743
batch 5428: loss 0.000376
batch 5429: loss 0.058791
batch 5430: loss 0.003020
batch 5431: loss 0.001195
batch 5432: loss 0.005747
batch 5433: loss 0.000786
batch 5434: loss 0.001017
batch 5435: loss 0.000028
batch 5436: loss 0.017871
batch 5437: loss 0.001826
batch 5438: loss 0.089276
batch 5439: loss 0.000349
batch 5440: loss 0.000292
batch 5441: loss 0.022380
batch 5442: loss 0.061080
batch 5443: loss 0.012528
batch 5444: loss 0.002362
batch 5445: loss 0.006290
batch 5446: loss 0.001046
batch 5447: loss 0.000167
batch 5448: loss 0.000143
batch 5449: loss 0.006584
batch 5450: loss 0.001237
batch 5451: loss 0.083046
batch 5452: loss 0.006413
batch 5453: loss 0.028368
batch 5454: loss 0.000503
batch 5455: loss 0.000127
batch 5456: 

batch 5735: loss 0.001392
batch 5736: loss 0.003915
batch 5737: loss 0.036234
batch 5738: loss 0.001135
batch 5739: loss 0.008232
batch 5740: loss 0.005344
batch 5741: loss 0.001965
batch 5742: loss 0.083403
batch 5743: loss 0.005629
batch 5744: loss 0.009521
batch 5745: loss 0.007718
batch 5746: loss 0.047678
batch 5747: loss 0.000313
batch 5748: loss 0.003199
batch 5749: loss 0.004162
batch 5750: loss 0.037810
batch 5751: loss 0.000051
batch 5752: loss 0.001153
batch 5753: loss 0.027714
batch 5754: loss 0.006847
batch 5755: loss 0.172004
batch 5756: loss 0.001926
batch 5757: loss 0.008667
batch 5758: loss 0.006689
batch 5759: loss 0.038392
batch 5760: loss 0.005975
batch 5761: loss 0.010691
batch 5762: loss 0.002220
batch 5763: loss 0.008201
batch 5764: loss 0.014383
batch 5765: loss 0.000490
batch 5766: loss 0.018113
batch 5767: loss 0.006264
batch 5768: loss 0.016701
batch 5769: loss 0.004805
batch 5770: loss 0.001988
batch 5771: loss 0.001911
batch 5772: loss 0.002138
batch 5773: 

In [6]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.990900
